# GCC's ssc-cdi : tomography pipeline for Cateretê v0.1
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/

## Imports

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import json
import sscCdi
import sscRaft

print(f'sscCdi version: {sscCdi.__version__}')
print(f'sscRaft version: {sscRaft.__version__}')

## Load dictionary

In [ ]:
dic = {}

In [ ]:
# dic = json.load(open('/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/inputs/CAT_tomo_inputs.json'))
# dic['processing_steps'] = {}
# dic['processing_steps']['read'] = True
# dic['processing_steps']['sort'] = True
# dic['processing_steps']['crop'] = True
# dic['processing_steps']['equalize2D'] = True
# dic['processing_steps']['unwrap'] = True
# dic['processing_steps']['alignment'] = True
# dic['processing_steps']['tomography'] = True
# dic['processing_steps']['equalize3D'] = True
# dic

# Run at Cluster

In [ ]:
# server = sscInstall.connect_server()  

In [ ]:
# json_filepath_path = sscCdi.cat_ptycho_processing.save_input_dictionary(dic)
# sscCdi.run_at_cluster(server,json_filepath_path,queue='cat',gpus=dic["GPUs"],cpus=dic["CPUs"],jobName='job',script_path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/ssc-cdi-0.5.1/bin/caterete_tomo.py",slurm_path = '/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/inputs/')

# Run Locally

## Load data

In [ ]:
dic["recon_method"]  = "ptycho" # ptycho or pwcdi
dic["contrast_type"] = "complex" # phase, magnitude or complex
dic["sinogram_path"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/glass_sphere/2023-05-16-15h38m_glass_sphere.hdf5"
# dic["sinogram_path"] = "/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/sampleb_3D/2023-07-17-16h57m_sampleb_3D.hdf5"

In [ ]:
dic = sscCdi.define_paths(dic)

In [ ]:
obj, angles = sscCdi.read_data(dic)
print(f"Object shape = {obj.shape} \t Number of angles: {angles.shape}")

In [ ]:
proj = np.sum(obj,axis=0)

In [ ]:
plt.figure()
plt.imshow(np.angle(proj[1200:2700,1200:3800]))

In [ ]:
# sscCdi.deploy_visualizer(obj,title='Original sinogram',cmap='gray')

In [ ]:
obj = obj[0::4,1200:2700,1200:3800]
angles = angles[0::4] 

In [ ]:
np.save(dic["cropped_sinogram_filepath"],obj)

## Sort data

In [ ]:
angles[:,0] = [ i for i in range (angles.shape[0])]

In [ ]:
obj.shape, angles.shape

In [ ]:
plt.figure()
plt.plot(angles[:,0], 'o')

In [ ]:
sscCdi.tomo_sort(dic,obj, angles)

In [ ]:
dic["ordered_object_filepath"]

In [ ]:
ordered_angles = '/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/sampleb_3D/temp/'+'2023-07-17-16h57m_sampleb_3D_complex_ordered_angles.npy'
ordered_angles = np.load(ordered_angles)

In [ ]:
0.7e-6*180/np.pi

In [ ]:
ordered_angles[112:120]

In [ ]:
sorted_data = np.load(dic["ordered_object_filepath"])
print(sorted_data.shape)
sscCdi.deploy_visualizer(sorted_data,type='phase',title="Ordered sinogram",cmap='gray')

In [ ]:
np.save(dic["cropped_sinogram_filepath"],sorted_data)

## Crop data

In [ ]:
dic["top_crop"]    = 1
dic["bottom_crop"] = 1
dic["left_crop"]   = 1
dic["right_crop"]  = 1

In [ ]:
sscCdi.tomo_crop(dic)

In [ ]:
cropped_data = np.load(dic["cropped_sinogram_filepath"])

In [ ]:
sscCdi.deploy_visualizer(cropped_data,type='phase',title="Cropped sinogram",cmap='gray')

## Alignment: Cross Correlation (CC) and Vertical Mass Fluctuation (VMF)

In [ ]:
aligned_data_CC = sscCdi.alignment_variance_field(cropped_data, fft_upsampling=10,return_common_valid_region=True, remove_null_borders = True)

In [ ]:
sscCdi.misc.deploy_visualizer(aligned_data_CC,type='phase',title='CC',cmap='gray',axis=0) # select axis

In [ ]:
aligned_data_VMF = sscCdi.alignment_vertical_mass_fluctuation(aligned_data_CC, use_phase_gradient = True, return_common_valid_region=True, remove_null_borders = True, plot = False) # if data is not equalized, phase gradient should be used

In [ ]:
sscCdi.misc.deploy_visualizer(np.angle(aligned_data_VMF),title='VMF',cmap='gray',axis=0) # select axis

In [ ]:
np.save(dic["pre_aligned_sinogram_filepath"],aligned_data_VMF) # select which prealigned dataset to save

## Unwrap

In [ ]:
dic["bad_frames_before_unwrap"] = []

""" Select data to be unwrapped """
data_to_unwrap = np.angle(cropped_data)
# data_to_unwrap = np.angle(aligned_data_CC)
# data_to_unwrap = np.angle(aligned_data_VMF)

In [ ]:
sscCdi.tomo_unwrap(dic, data_to_unwrap)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["unwrapped_sinogram_filepath"]),title="Unwrapped sinogram",cmap='gray')

## 2D Equalization

In [ ]:
dic["bad_frames_before_equalization"] = []

dic["CPUs"] = 32

dic["equalize_invert"] = True # invert phase shift signal from negative to positive
dic["equalize_ROI"] = [0,1400,0,200] # region of interest of null region around the sample used for phase ramp and offset corrections
dic["equalize_remove_phase_gradient"] = True  # if empty and equalize_ROI = [], will subtract best plane fit from whole image
dic["equalize_remove_phase_gradient_iterations"] = 10
dic["equalize_local_offset"] = False # remove offset of each frame from the mean of ROI 
dic["equalize_set_min_max"]= [] # [minimum,maximum] threshold values for whole volume
dic["equalize_non_negative"] = False # turn any remaining negative values to zero


In [ ]:
sinogram_to_equalize = np.load(dic["unwrapped_sinogram_filepath"])
sscCdi.tomo_equalize(dic,sinogram_to_equalize)

In [ ]:
sscCdi.misc.plot_volume_histogram(np.load(dic["equalized_sinogram_filepath"]))

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram",cmap='gray',limts)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram",cmap='gray',axis=1,aspect_ratio='auto')#,limits=(0,0.5))

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram",cmap='gray',axis=2,aspect_ratio='auto')#,limits=(0,0.5))

## Alignment (wiggle) 

In [ ]:
dic["wiggle_sinogram_selection"] = dic["equalized_sinogram_filepath"]
dic["step_percentage"] = 0 # need to project irregular angle steps to a regular grid?

In [ ]:
sscCdi.preview_angle_projection(dic)

In [ ]:
dic["project_angles_to_regular_grid"] = True 
dic["bad_frames_before_wiggle"] = [] 
dic["wiggle_reference_frame"] = 0 

In [ ]:
dic = sscCdi.tomo_alignment(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["wiggle_sinogram_filepath"]),title="Aligned sinogram",cmap='gray',axis=1)

## Tomography: select dictionary according to tomographic method
- for details on the tomography algorithm inputs, see documentation at https://gcc.lnls.br/wiki/docs/ssc-raft/reconstructions/

In [ ]:
dic['using_wiggle'] = True

dic['automatic_regularization'] = 0 # skip if 0; regularization between 0 and 1 to enhance borders prior to recon (https://www.sciencedirect.com/science/article/pii/S2590037419300883?via%3Dihub)

# dic["algorithm_dic"] = { # if FBP: filtered back-projection
#     'algorithm': "FBP",
#     'gpu': [0,1],
#     'filter': 'lorentz', # 'gaussian','lorentz','cosine','rectangle'
#     'regularization': 0.1, # 0 <= regularization <= 1; use for smoothening
# }

dic["algorithm_dic"] = { # if eEM: emission expectation maximization
    'algorithm': "EM",
    'gpu': [0,1],
    'regularization': 0.0001,
    'method': 'eEM', 
    'niterations': [20,0,0,0], # [global iterations, iterations EMTV, iterations2 EMTV, Cone-beam integration points]
    'epsilon': 1e-15, #for EMTV only
    'blocksize': 20, # blocks for parallelization
}

In [ ]:
# sinogram = np.load(dic["equalized_sinogram_filepath"])
sinogram = np.load(dic["wiggle_sinogram_filepath"])
recon3D = sscCdi.tomo_recon(dic,sinogram)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["reconstruction_filepath"]),title="Recon",cmap='gray',axis=0)#,limits=(-1,1))

## 3D Equalization

In [ ]:
dic["tomo_remove_outliers"] = 0
dic["tomo_threshold"] = 20.0
dic["tomo_local_offset"] = []
dic["tomo_mask"] = []

In [ ]:
sscCdi.tomo_equalize3D(dic)

In [ ]:
sscCdi.deploy_visualizer(np.load(dic["eq_reconstruction_filepath"]),title="Equalized Recon",cmap='gray',axis=0)

In [ ]:
equalized_tomogram = np.load(dic["eq_reconstruction_filepath"])
sscCdi.plot_histograms(recon3D, equalized_tomogram,bins=300)